# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 19 2022 1:21PM,253384,10,0086257046,ISDIN Corporation,Released
1,Dec 19 2022 1:21PM,253384,10,0086257047,ISDIN Corporation,Released
2,Dec 19 2022 1:21PM,253384,10,0086257049,ISDIN Corporation,Released
3,Dec 19 2022 1:20PM,253383,21,636497,"NBTY Global, Inc.",Released
4,Dec 19 2022 1:19PM,253382,21,636496,"NBTY Global, Inc.",Released
5,Dec 19 2022 1:01PM,253379,10,CTF0011548,"Citieffe, Inc.",Released
6,Dec 19 2022 1:01PM,253379,10,CTF0011549,"Citieffe, Inc.",Released
7,Dec 19 2022 1:00PM,253378,10,SO94473,"Senseonics, Incorporated",Released
8,Dec 19 2022 1:00PM,253378,10,SO94474,"Senseonics, Incorporated",Released
9,Dec 19 2022 12:47PM,253377,15,8922916,Tripoint Therapeutics LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,253378,Released,2
24,253379,Released,2
25,253382,Released,1
26,253383,Released,1
27,253384,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253378,NaN,NaN,2.0
253379,NaN,NaN,2.0
253382,NaN,NaN,1.0
253383,NaN,NaN,1.0
253384,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253332,1.0,4.0,8.0
253340,5.0,14.0,0.0
253344,0.0,0.0,6.0
253358,0.0,21.0,1.0
253359,0.0,7.0,16.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253332,1,4,8
253340,5,14,0
253344,0,0,6
253358,0,21,1
253359,0,7,16


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253332,1,4,8
1,253340,5,14,0
2,253344,0,0,6
3,253358,0,21,1
4,253359,0,7,16


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253332,1,4,8
1,253340,5,14,
2,253344,,,6
3,253358,,21,1
4,253359,,7,16


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 19 2022 1:21PM,253384,10,ISDIN Corporation
3,Dec 19 2022 1:20PM,253383,21,"NBTY Global, Inc."
4,Dec 19 2022 1:19PM,253382,21,"NBTY Global, Inc."
5,Dec 19 2022 1:01PM,253379,10,"Citieffe, Inc."
7,Dec 19 2022 1:00PM,253378,10,"Senseonics, Incorporated"
9,Dec 19 2022 12:47PM,253377,15,Tripoint Therapeutics LLC
10,Dec 19 2022 12:45PM,253376,10,Yusen – 3D Matrix
11,Dec 19 2022 12:36PM,253375,19,"Innogenix, LLC"
38,Dec 19 2022 12:17PM,253374,19,"GUSA Granules USA, Inc."
40,Dec 19 2022 12:06PM,253373,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 19 2022 1:21PM,253384,10,ISDIN Corporation,,,3
1,Dec 19 2022 1:20PM,253383,21,"NBTY Global, Inc.",,,1
2,Dec 19 2022 1:19PM,253382,21,"NBTY Global, Inc.",,,1
3,Dec 19 2022 1:01PM,253379,10,"Citieffe, Inc.",,,2
4,Dec 19 2022 1:00PM,253378,10,"Senseonics, Incorporated",,,2
5,Dec 19 2022 12:47PM,253377,15,Tripoint Therapeutics LLC,,,1
6,Dec 19 2022 12:45PM,253376,10,Yusen – 3D Matrix,,,1
7,Dec 19 2022 12:36PM,253375,19,"Innogenix, LLC",,11,16
8,Dec 19 2022 12:17PM,253374,19,"GUSA Granules USA, Inc.",,,2
9,Dec 19 2022 12:06PM,253373,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 19 2022 1:21PM,253384,10,ISDIN Corporation,3,,
1,Dec 19 2022 1:20PM,253383,21,"NBTY Global, Inc.",1,,
2,Dec 19 2022 1:19PM,253382,21,"NBTY Global, Inc.",1,,
3,Dec 19 2022 1:01PM,253379,10,"Citieffe, Inc.",2,,
4,Dec 19 2022 1:00PM,253378,10,"Senseonics, Incorporated",2,,
5,Dec 19 2022 12:47PM,253377,15,Tripoint Therapeutics LLC,1,,
6,Dec 19 2022 12:45PM,253376,10,Yusen – 3D Matrix,1,,
7,Dec 19 2022 12:36PM,253375,19,"Innogenix, LLC",16,11,
8,Dec 19 2022 12:17PM,253374,19,"GUSA Granules USA, Inc.",2,,
9,Dec 19 2022 12:06PM,253373,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 19 2022 1:21PM,253384,10,ISDIN Corporation,3,,
1,Dec 19 2022 1:20PM,253383,21,"NBTY Global, Inc.",1,,
2,Dec 19 2022 1:19PM,253382,21,"NBTY Global, Inc.",1,,
3,Dec 19 2022 1:01PM,253379,10,"Citieffe, Inc.",2,,
4,Dec 19 2022 1:00PM,253378,10,"Senseonics, Incorporated",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 19 2022 1:21PM,253384,10,ISDIN Corporation,3.0,NaN,NaN
1,Dec 19 2022 1:20PM,253383,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Dec 19 2022 1:19PM,253382,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Dec 19 2022 1:01PM,253379,10,"Citieffe, Inc.",2.0,NaN,NaN
4,Dec 19 2022 1:00PM,253378,10,"Senseonics, Incorporated",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 19 2022 1:21PM,253384,10,ISDIN Corporation,3.0,0.0,0.0
1,Dec 19 2022 1:20PM,253383,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Dec 19 2022 1:19PM,253382,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Dec 19 2022 1:01PM,253379,10,"Citieffe, Inc.",2.0,0.0,0.0
4,Dec 19 2022 1:00PM,253378,10,"Senseonics, Incorporated",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2533670,19.0,18.0,5.0
15,760094,18.0,28.0,0.0
19,1266810,28.0,15.0,1.0
21,506765,2.0,0.0,0.0
22,253373,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2533670,19.0,18.0,5.0
1,15,760094,18.0,28.0,0.0
2,19,1266810,28.0,15.0,1.0
3,21,506765,2.0,0.0,0.0
4,22,253373,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,19.0,18.0,5.0
1,15,18.0,28.0,0.0
2,19,28.0,15.0,1.0
3,21,2.0,0.0,0.0
4,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,19.0
1,15,Released,18.0
2,19,Released,28.0
3,21,Released,2.0
4,22,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,21,22
Status,,,,,
Completed,5.0,0.0,1.0,0.0,0.0
Executing,18.0,28.0,15.0,0.0,0.0
Released,19.0,18.0,28.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,21,22
0,Completed,5.0,0.0,1.0,0.0,0.0
1,Executing,18.0,28.0,15.0,0.0,0.0
2,Released,19.0,18.0,28.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,21,22
0,Completed,5.0,0.0,1.0,0.0,0.0
1,Executing,18.0,28.0,15.0,0.0,0.0
2,Released,19.0,18.0,28.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()